In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 23:04 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 22:47 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 23:04 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 22:47 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal data.tsv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/data.tsv': File exists


In [4]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 31.352 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.855 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 3.287 seconds


In [11]:
%%hive
DROP TABLE IF EXISTS cant_let_clav;
CREATE TABLE cant_let_clav AS
SELECT f.v1, f.v2, COUNT(*)
FROM
(SELECT a.c AS v1, a.k AS v2
FROM  
(SELECT c2, k 
FROM t0 
LATERAL VIEW explode(c3) t0 AS k, v) a
LATERAL VIEW explode(a.c2) a AS c) f
GROUP BY f.v1, f.v2;

DROP TABLE IF EXISTS cant_let_clav;
OK
Time taken: 0.023 seconds
CREATE TABLE cant_let_clav AS
SELECT f.v1, f.v2, COUNT(*)
FROM
(SELECT a.c AS v1, a.k AS v2
FROM  
(SELECT c2, k 
FROM t0 
LATERAL VIEW explode(c3) t0 AS k, v) a
LATERAL VIEW explode(a.c2) a AS c) f
GROUP BY f.v1, f.v2;
Query ID = root_20200218232304_b35be194-07fa-458e-8f18-9c34d9228330
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0037, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0037/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0037
Hadoop job information for Stage-1: number of mappers:

In [12]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM cant_let_clav;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM cant_let_clav;
Query ID = root_20200218232351_b471b370-0302-4688-9fe8-740b68774dac
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0038, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0038/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0038
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 23:24:10,885 Stage-1 map = 0%,  reduce = 0%
2020-02-18 23:24:22,182 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.48 sec
MapReduce Total cumulative CPU time: 3 seconds 480 msec
Ended Job = job_1582053079980_0038
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-stagin

In [13]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [14]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [15]:
!cat output/000000_0

a,aaa,5
a,bbb,7
a,ccc,13
a,ddd,13
a,eee,7
a,fff,10
a,ggg,8
a,hhh,8
a,iii,7
a,jjj,10
b,aaa,4
b,bbb,7
b,ccc,5
b,ddd,7
b,eee,5
b,fff,8
b,ggg,4
b,hhh,7
b,iii,7
b,jjj,5
c,aaa,5
c,bbb,4
c,ccc,12
c,ddd,9
c,eee,6
c,fff,8
c,ggg,7
c,hhh,7
c,iii,8
c,jjj,8
d,aaa,4
d,bbb,6
d,ccc,7
d,ddd,5
d,eee,6
d,fff,8
d,ggg,6
d,hhh,4
d,iii,9
d,jjj,8
e,aaa,3
e,bbb,8
e,ccc,9
e,ddd,7
e,eee,7
e,fff,9
e,ggg,4
e,hhh,4
e,iii,8
e,jjj,3
f,aaa,8
f,bbb,10
f,ccc,13
f,ddd,17
f,eee,11
f,fff,11
f,ggg,9
f,hhh,10
f,iii,10
f,jjj,12
g,aaa,3
g,bbb,3
g,ccc,6
g,ddd,5
g,eee,4
g,fff,5
g,ggg,4
g,hhh,3
g,iii,4
g,jjj,6
